# For assigning chemical info and partial charges to PDB structures (saves to well-defined SDF files)

## Imports

In [1]:
# Generic Imports
from collections import defaultdict

# Logging
import logging
from polymerist.genutils.logutils.IOHandlers import LOG_FORMATTER

logging.basicConfig(
    level=logging.WARNING,
    format =LOG_FORMATTER._fmt,
    datefmt=LOG_FORMATTER.datefmt,
    force=True
)

# File I/O
from pathlib import Path
from tqdm.notebook import tqdm
from rich.progress import Progress, track # TODO : convert from tqdm

# Cheminformatics
from openff.toolkit.topology import Topology, Molecule

# Static Paths
MONO_DIR = Path('monomer_fragments')
PDB_DIR  = Path('pdb_files_new/trimers')

/home/timber/miniconda3/envs/polymerist-env/lib/python3.11/site-packages/numpy/core/getlimits.py:500: UserWarning: The value of the smallest subnormal for <class 'numpy.float64'> type is zero.
  setattr(self, word, getattr(machar, word).flat[0])
/home/timber/miniconda3/envs/polymerist-env/lib/python3.11/site-packages/numpy/core/getlimits.py:89: UserWarning: The value of the smallest subnormal for <class 'numpy.float64'> type is zero.
  return self._float_to_str(self.smallest_subnormal)
/home/timber/miniconda3/envs/polymerist-env/lib/python3.11/site-packages/numpy/core/getlimits.py:500: UserWarning: The value of the smallest subnormal for <class 'numpy.float32'> type is zero.
  setattr(self, word, getattr(machar, word).flat[0])
/home/timber/miniconda3/envs/polymerist-env/lib/python3.11/site-packages/numpy/core/getlimits.py:89: UserWarning: The value of the smallest subnormal for <class 'numpy.float32'> type is zero.
  return self._float_to_str(self.smallest_subnormal)


## Obtaining complete PDB-monomer pairs for iteration

In [2]:
chem_paths = defaultdict(defaultdict)

for chem_mono in MONO_DIR.iterdir():
    chemistry = chem_mono.stem
    chem_pdb  = PDB_DIR / chemistry

    has_pdb  = set(path.stem for path in chem_pdb.glob( '*.pdb'))
    has_mono = set(path.stem for path in chem_mono.glob('*.json'))

    for valid_mol_name in (has_pdb & has_mono):
        pdb_path  = chem_pdb  / f'{valid_mol_name}.pdb'
        mono_path = chem_mono / f'{valid_mol_name}.json'

        chem_paths[chemistry][valid_mol_name] = (pdb_path, mono_path)

## Parameterizing and saving to SDF, recording mols where match fails

In [3]:
from polymerist.openfftools import topology
from polymerist.openfftools.pcharge import MolCharger

from polymerist.monomers import MonomerGroup
from polymerist.residues.partition import partition

# catch annoying warnings
import warnings 
from openff.toolkit.utils.exceptions import IncorrectNumConformersWarning
warnings.catch_warnings(record=True)
warnings.filterwarnings('ignore', category=IncorrectNumConformersWarning)

group_blacklist = [
    'poly(4-N-(4-aminophenyl)-4-N-[4-(2,4,6-tritert-butylphenoxy)phenyl]benzene-1,4-diamine' # for some reason, these compounds are unable to be parameterized in a reasonable amount of time
]


topo_dir = Path('Topologies_new')
topo_dir.mkdir(exist_ok=True)

charge_method = 'Espaloma-AM1-BCC'
charger = MolCharger.subclass_registry[charge_method]()


unmatched_pdb_mols = defaultdict(defaultdict)
cancelled = defaultdict(list)
for chemistry, path_dict in chem_paths.items():
    chem_dir = topo_dir / chemistry
    chem_dir.mkdir(exist_ok=True)

    progress = tqdm(path_dict.items())
    for mol_name, (pdb_path, mono_path) in progress:
        progress.set_description_str(f'{chemistry} : {mol_name}')
        if any(group_name in mol_name for group_name in group_blacklist):
            progress.set_postfix_str('Cancelled')
            cancelled[chemistry].append(mol_name)
            continue

        try:
            progress.set_postfix_str('Loading files')
            monogrp = MonomerGroup.from_file(mono_path)
            offtop = Topology.from_pdb(pdb_path, _custom_substructures=monogrp.monomers)
            progress.set_postfix_str('Partitioning')
            was_partitioned = partition(offtop)
            assert(was_partitioned)

            progress.set_postfix_str('Assigning Charges')
            offmol = topology.get_largest_offmol(offtop)
            offmol.name = mol_name
            cmol = charger.charge_molecule(offmol)

            progress.set_postfix_str('Saving SDF')
            offtop = cmol.to_topology()
            sdf_path = chem_dir / f'{mol_name}.sdf'
            topology.topology_to_sdf(sdf_path, offtop)
        except Exception as e:
            print(f'{mol_name} : {e}')
            unmatched_pdb_mols[str(e)][chemistry] = mol_name

  0%|          | 0/36 [00:00<?, ?it/s]

/home/timber/miniconda3/envs/polymerist-env/lib/python3.11/site-packages/dgl/heterograph.py:92: DGLWarning: Recommend creating graphs by `dgl.graph(data)` instead of `dgl.DGLGraph(data)`.
  dgl_warning(


  0%|          | 0/3 [00:00<?, ?it/s]

  0%|          | 0/43 [00:00<?, ?it/s]

  0%|          | 0/26 [00:00<?, ?it/s]

In [ ]:
unmatched_pdb_mols

## Probing PDB mismatches

In [ ]:
unmatched_pdb_mols['urethane']['urethane_13']

In [ ]:
for mol_name, mol in unmatched_pdb_mols['urethane'].items():
    mol.name = mol_name

mols_by_len = sorted(unmatched_pdb_mols['urethane'].values(), key=lambda mol : mol.n_atoms)

In [ ]:
mol = mols_by_len[-1]

rdmol = mol.to_rdkit()
for i, atom in enumerate(mol.atoms):
    if not atom.metadata['already_matched']:
        print(atom.metadata)
        rdmol.GetAtomWithIdx(i).SetProp('atomNote', 'unmatched')

display(rdmol)